In [1]:
import pandas as pd
from dotenv import load_dotenv

from pydantic import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv(
    dotenv_path="/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env"
)
# from trader.fetch_requests import *
# from trader.prepare_requests import *
from api_helpers.clients import get_betfair_client, get_s3_client, get_postgres_client

postgres_client = get_postgres_client()

2025-08-22T07:03:03Z | INFO - Logging configuration initialized with level: INFO


In [5]:
df = pd.read_csv('~/Desktop/test.csv')

In [6]:
df.columns

Index(['race_id', 'horse_id', 'horse_name', 'age', 'race_time', 'race_date',
       'race_title', 'race_type', 'race_class', 'distance', 'distance_yards',
       'distance_meters', 'distance_kilometers', 'conditions', 'going',
       'betfair_win_sp', 'number_of_runners', 'hcap_range', 'age_range',
       'surface', 'total_prize_money', 'first_place_prize_money', 'course_id',
       'course', 'data_type'],
      dtype='object')

In [6]:
df[
    [
        "horse_name",
        "official_rating",
        "race_id",
        "horse_id",
        "race_date",
        "race_class",
        "race_type",
        "distance",
        "going",
        "surface",
        "course",
        "rating",
        "speed_figure",
        "unique_id",
    ]
]

,horse_name,official_rating,race_id,horse_id,race_date,race_class,race_type,distance,going,surface,course,rating,speed_figure,unique_id
0,Boom The Groom,62.0,813646,99370,2022-06-21,6,Flat,6f,Good To Firm,turf,Brighton,67,61,023889d994a650fe57c48e6d8a67678e1f12f51e92a964...
1,Boom The Groom,62.0,812445,99370,2022-06-07,6,Flat,6f,Good To Soft,turf,Brighton,66,46,a26d25841959cb254b9ffdff2f6444bb125fbac8f14e37...
2,Boom The Groom,64.0,811183,99370,2022-05-24,6,Flat,5f,Good To Soft,turf,Bath,56,48,d36b8702e6d37dfc9efc4198f26da6098593e0c855de88...
3,Boom The Groom,67.0,808547,99370,2022-04-25,5,Flat,5f,Good To Firm,turf,Windsor,41,29,36f2a95b661bd384165121ba9ce47060d81bd13b6e07eb...
4,Boom The Groom,70.0,804316,99370,2022-03-07,5,Flat,6f,Standard,tapeta,Wolverhampton,44,36,0d3f0d39c8315ddb7d5fe3032a12f7818f64b776df469d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Desert Team,73.0,811288,170057,2022-05-28,5,Flat,5f,Good To Firm,turf,Catterick,87,64,4a277c92154597700a0ccc3d21d09f06e88644472f5554...
123,Desert Team,73.0,808701,170057,2022-05-02,5,Flat,5f,Good,turf,Beverley,76,62,f4edb217d990770b6fac91d45f48a3fadc3f7f1f0451b9...
124,Desert Team,NaN,807712,170057,2022-04-23,5,Flat,5f,Standard,tapeta,Wolverhampton,79,73,6715faa3af6aca2de6236742e732a04238211dc0d5ca2e...
125,Desert Team,NaN,806511,170057,2022-04-06,5,Flat,6f,Standard To Slow,polytrack,Kempton,70,45,659c1089f94edcfc1dca17ed9cfd810e4dd8d3970048a2...
